In [1]:
%run lib/scrape_govuk.ipynb
metadata = scrape('https://www.gov.uk/government/statistics/immigration-statistics-october-to-december-2017-data-tables')

import pyexcel
from io import BytesIO
import time

ods_files = [f for f in metadata['files'] if f['type'] == 'ODS']

out = Path('out')
out.mkdir(exist_ok=True)

def load_sheet(ods_file_info, sheet_name):
    display(f"Loading '{ods_file_info['title']}', sheet '{sheet_name}'...")
    start = time.time()
    ods_file = BytesIO(session.get(ods_file_info['url']).content)
    sheet = pyexcel.get_sheet(
        file_content=ods_file,
        file_type='ods',
        sheet_name=sheet_name,
        library='pyexcel-ods3'
    )
    display(f"in {int(time.time() - start)}s")
    return sheet

def load_book(ods_file_info):
    display(f"Loading '{ods_file_info['title']}'...")
    start = time.time()
    ods_file = BytesIO(session.get(ods_file_info['url']).content)
    book = pyexcel.get_book(
        file_content=ods_file,
        file_type='ods',
        library='pyexcel-ods3'
    )
    display(f"in {int(time.time() - start)}s")
    return book

for ods_file_info in ods_files:
    if ods_file_info['title'] == 'Entry clearance visas granted outside the UK data tables immigration statistics October to December 2017 volume 1':
        book = load_book(ods_file_info)
        data = book['vi_05']
        display(f"Processing '{ods_file_info['title']}', sheet 'vi_05'")
        %run "Entry clearance visas granted by country of nationality(Vi_05).ipynb"
        entry_1 = tidy.copy()
        data = book['vi_01_q']
        display(f"Processing '{ods_file_info['title']}', sheet 'v1_01_q'")
        %run "Entry clearance visa applications and resolution by category ( vi_01_q).ipynb"
        entry_2 = tidy.copy()
        entry_1.rename(columns={'Year': 'Period'}, inplace=True)
        entry_1.replace(
            {'Period': r'$([0-9]{4})^'}, {'Period': r'year/\1'}, regex=True, inplace=True
        )
        for col in ['Type', 'Broad category', 'Applicant type', 'Category', 'Resolution']:
            entry_1[col] = 'All'
        # Todo: "World" or "Not UK"?
        entry_2['Country of nationality'] = 'World'
        pd.concat([entry_1, entry_2], sort=True).to_csv(out / 'entry_visas.csv', index=False)

"Loading 'Entry clearance visas granted outside the UK data tables immigration statistics October to December 2017 volume 1'..."

'in 19s'

"Processing 'Entry clearance visas granted outside the UK data tables immigration statistics October to December 2017 volume 1', sheet 'vi_05'"

,Geographical region,Country of nationality,Year,Value


['Asia Central',
 'America North',
 'Montserrat (British)',
 'EU Other',
 'Ireland',
 'Korea (North)',
 'Africa North',
 'British overseas citizens',
 'EU 8',
 'Curacao',
 'St. Martin (French Part)',
 'United States',
 'St. Pierre and Miquelon',
 'Spain',
 'Refugee',
 'Palau',
 'Cocos (Keeling) Islands',
 'St. Vincent and the Grenadines',
 'Bonaire, Sint Eustatius and Saba',
 'Surinam',
 'St. Helena (British)',
 'Europe Other',
 'Cook Islands',
 'Occupied Palestinian Territories',
 'Asia South East',
 'Kiribati',
 'Samoa',
 'St. Maarten (Dutch Part)',
 'Turks and Caicos Islands (British)',
 'Central African Republic',
 'Asia South',
 'Burkina',
 'Guinea-Bissau',
 'Korea (South)',
 'Other',
 'Faroe Islands',
 'Pitcairn Islands (British)',
 'Falkland Islands (British)',
 'Other and unknown',
 'Wallis and Futuna',
 'Kosovo',
 'Cambodia',
 'Virgin Islands (US)',
 'Netherlands Antilles',
 'St. Lucia',
 'Cyprus',
 'America Central and South',
 'Cayman Islands (British)',
 'Christmas Island',

"Processing 'Entry clearance visas granted outside the UK data tables immigration statistics October to December 2017 volume 1', sheet 'v1_01_q'"